In [323]:
import geopandas
import networkx as nx
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

#PERIMETER = '_paper2_example'
PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
outputs_path_accessibility = os.path.join(data_directory, 'outputs', '_accessibility_debug')

CRS_internal = 2056      # for Zurich

In [324]:
population_sample = 0.05
#statent_id = 68372518
statent_id = 67912473
distance_limit = 70*1000
destinations_sample = 0.05
cycling_speed_kmh = 18
walking_speed_kmh = 1.34 * 3.6
access_egress_detour_factor = 2
departure_time = datetime.datetime(2024, 2, 22, 18, 00)
base_travel_time = 5*60

rebuild_inputs = True
STATE = 'after'

if STATE=='before':
    ln_desc = KEY_LANES_DESCRIPTION
else:
    ln_desc = KEY_LANES_DESCRIPTION_AFTER

In [325]:
if 0:
    r5_transit_network = r5py.TransportNetwork(
        os.path.join(outputs_path_accessibility, 'before_oneway_links_default.osm.pbf'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip'),
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip'),
    )

In [326]:
# Load street graph
if 0:
    G = snman.io.load_street_graph(
        os.path.join(outputs_path, 'G_edges.gpkg'),
        os.path.join(outputs_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )

In [327]:
if rebuild_inputs:
    G_modes = {}
    L_modes = {}

In [328]:
# create mode graphs for other modes
if rebuild_inputs:
    for mode in [MODE_CYCLING, MODE_FOOT]:
        print(f'Make street and lane graphs for {mode}')
        G_modes[mode] = copy.deepcopy(G)
        snman.street_graph.filter_lanes_by_modes(
            G_modes[mode], [mode], lane_description_key=ln_desc
        )
        L_modes[mode] = snman.lane_graph.create_lane_graph(
            G_modes[mode], lanes_attribute=ln_desc
        )

Make street and lane graphs for cycling
Make street and lane graphs for foot


In [329]:
if rebuild_inputs:
    # load the new car street graph created from MATSim output
    G_modes[MODE_PRIVATE_CARS] = snman.io.load_street_graph(
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_internal,
        unstringify_attributes={'lanes': snman.space_allocation.space_allocation_from_string}
    )
    
    # we must avoid that the start/end point of trips are matched onto nodes that not accessible by car
    # therefore, we reduce the graph only to those links and nodes that are accessible by car
    snman.street_graph.filter_lanes_by_modes(
        G_modes[MODE_PRIVATE_CARS], [MODE_PRIVATE_CARS], lane_description_key='lanes'
    )

In [330]:
if rebuild_inputs:
    L_modes[MODE_PRIVATE_CARS] = snman.lane_graph.create_lane_graph(
        G_modes[MODE_PRIVATE_CARS], lanes_attribute='lanes', cast_attributes={'matsim_tt_cars': 'car_18:00'}
    )

In [331]:
if rebuild_inputs:
    # Import Statent grid
    statent = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
    )
    statent.rename(columns={'RELI': 'id'}, inplace=True)
    statent

In [332]:
if rebuild_inputs:
    # read statpop that has been pre-joined with statent
    statpop_with_statent_ids = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
    )
    
    statpop_with_statent_ids.head()

In [333]:
# filter statpop for residents within the given origin statent cell
statpop_filtered = statpop_with_statent_ids.query(f'statent_id == {statent_id}')
statpop_filtered = statpop_filtered.sample(frac=population_sample, random_state=0)
statpop_filtered

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
7921339,2017,1963,2,1,-2,54,-2,899839562,POINT (2679130.001 1247261.001),23550.0,67912473.0,49.203130
9180957,2017,1952,1,1,-2,65,-2,899839587,POINT (2679102.001 1247322.001),23550.0,67912473.0,22.091913
12659585,2017,1950,2,2,-2,67,-2,899839528,POINT (2679103.001 1247285.001),23550.0,67912473.0,15.296061


In [334]:
# make a light version of statpop
statpop_reduced = statpop_filtered.reset_index()[[
    'record', 'age', 'sex', 'maritalstatus', 'residencepermit', 'residentpermit', 'statent_id', 'geometry'
]]
statpop_reduced.set_index('record', inplace=True)
statpop_reduced

,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry
record,,,,,,,
7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001)
9180957,65,1,1,-2,-2,67912473.0,POINT (2679102.001 1247322.001)
12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001)


In [335]:
# filter statent for cells within a distance limit from the origin
origin = statent.set_index('id').geometry[statent_id]
origins = gpd.GeoDataFrame(
    {
        "id": [statent_id],
        "geometry": [origin]
    },
    crs="EPSG:2056",
)
statent_filtered = statent[statent['geometry'].within(
    origin.buffer(distance_limit)
)]
statent_filtered = gpd.GeoDataFrame(statent_filtered)
statent_filtered

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR1,VOLLZEITAEQ_SEKTOR1_FRAUEN,VOLLZEITAEQ_SEKTOR1_MAENNER,VOLLZEITAEQ_SEKTOR2,VOLLZEITAEQ_SEKTOR2_FRAUEN,VOLLZEITAEQ_SEKTOR2_MAENNER,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry
0,2021,2023,2632200,1247300,63222473,4,0,4,4,7,...,0.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,0.0,POINT (2632200.000 1247300.000)
1,2021,2023,2632300,1248100,63232481,4,4,0,0,4,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2632300.000 1248100.000)
2,2021,2023,2632400,1247300,63242473,4,0,0,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632400.000 1247300.000)
3,2021,2023,2632400,1247500,63242475,4,0,4,0,4,...,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,POINT (2632400.000 1247500.000)
4,2021,2023,2632500,1247200,63252472,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632500.000 1247200.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58739,2021,2023,2728300,1222500,72832225,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728300.000 1222500.000)
58740,2021,2023,2728300,1222900,72832229,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,POINT (2728300.000 1222900.000)
58741,2021,2023,2728400,1222700,72842227,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728400.000 1222700.000)
58742,2021,2023,2728500,1222500,72852225,4,4,0,0,4,...,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2728500.000 1222500.000)


In [336]:
# make a sample of statpop cells but inlcude the origin cell in every case
statent_filtered = pd.concat([
    statent_filtered[statent_filtered['id'] == statent_id],
    statent_filtered[statent_filtered['id'] != statent_id].sample(frac=destinations_sample, random_state=0)
    ])

In [337]:
# match statent points to lanegraph nodes
# TODO: do this only once

for mode in [MODE_PRIVATE_CARS, MODE_CYCLING, MODE_FOOT]:

    # match statent points to nearest nodes
    nodes = snman.oxc.nearest_nodes(
        L_modes[mode],
        list(map(lambda geom: geom.x, statent_filtered.geometry)),
        list(map(lambda geom: geom.y, statent_filtered.geometry)),
        return_dist=True
    )

    statent_filtered[[f'closest_node_{mode}', f'access_egress_{mode}']] = list(zip(*nodes))

statent_filtered

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry,closest_node_private_cars,access_egress_private_cars,closest_node_cycling,access_egress_cycling,closest_node_foot,access_egress_foot
23550,2021,2023,2679100,1247300,67912473,6,0,0,6,59,...,35.135831,28.200000,6.935831,POINT (2679100.000 1247300.000),50608.0,28.931762,43277.0,28.931762,43277.0,28.931762
6655,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,0.000000,0.000000,0.000000,POINT (2653100.000 1239900.000),24514.0,1358.115708,50566.0,241.644861,50566.0,241.644861
5827,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,4.000000,4.000000,0.000000,POINT (2650500.000 1245800.000),53053.0,91.026213,51468.0,91.026213,51468.0,91.026213
38622,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,4.000000,4.000000,4.000000,POINT (2690900.000 1282700.000),49794.0,1371.805307,24747.0,31.243944,24747.0,31.243944
1399,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,6.989673,5.116902,4.000000,POINT (2637700.000 1255500.000),33074.0,51.306988,57357.0,51.306988,57357.0,51.306988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25346,2021,2023,2680800,1248500,68082485,4,0,0,4,4,...,4.000000,4.000000,4.000000,POINT (2680800.000 1248500.000),43114.0,45.950393,75309.0,45.952784,75309.0,45.952784
56523,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,4.000000,0.000000,4.000000,POINT (2714500.000 1258000.000),18198.0,1453.855424,37228.0,1352.131870,37228.0,1352.131870
44234,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,4.000000,4.000000,4.000000,POINT (2696900.000 1228000.000),16756.0,25.600192,4735.0,25.600192,4735.0,25.600192
25276,2021,2023,2680700,1259300,68072593,4,4,0,0,4,...,0.000000,0.000000,0.000000,POINT (2680700.000 1259300.000),30881.0,130.510729,29692.0,130.510729,29692.0,130.510729


In [338]:
tt_matrices = {}

In [339]:
# calculate transit travel time matrix using R5
tt_computer_transit = r5py.TravelTimeMatrixComputer(
    r5_transit_network,
    transport_modes=[r5py.TransportMode.TRANSIT],
    origins=origins,
    destinations=statent_filtered,
    departure=departure_time,
    snap_to_network=True
)
tt_matrices[MODE_TRANSIT] = tt_computer_transit.compute_travel_times()
tt_matrices[MODE_TRANSIT].rename(columns={
    'travel_time': 'travel_time_transit',
    'from_id': 'from_cell',
    'to_id': 'to_cell'
}, inplace=True)
# convert from minutes to seconds
tt_matrices[MODE_TRANSIT]['travel_time_transit'] *= 60

tt_matrices[MODE_TRANSIT]

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
C:\Users\lballo\AppData\Local\miniconda3\envs\snman\lib\site-packages\r5py\r5\base_travel_time_matrix_computer.py:152: RuntimeWarning: Some destination points could not be snapped to the street network
  warnings.warn(


,from_cell,to_cell,travel_time_transit
0,67912473,67912473,0.0
1,67912473,65312399,NaN
2,67912473,65052458,6240.0
3,67912473,69092827,7020.0
4,67912473,63772555,NaN
...,...,...,...
2858,67912473,68082485,1620.0
2859,67912473,71452580,6900.0
2860,67912473,69692280,5040.0
2861,67912473,68072593,4680.0


In [340]:
# calculate shortest paths to all destinations in networkx
for mode in [MODE_PRIVATE_CARS, MODE_FOOT, MODE_CYCLING]:

    print(mode)

    origin_cell = statent_filtered[statent_filtered['id'] == statent_id]
    origin_node = min(origin_cell[f'closest_node_{mode}'])
    access_cost = min(origin_cell[f'access_egress_{mode}'])
    print(origin_node, access_cost)
    
    if mode == MODE_PRIVATE_CARS:
        weight = 'matsim_tt_cars'
    else:
        weight = f'cost_{mode}'
    
    # calculate cost to all other nodes
    dijkstra_path_costs = nx.single_source_dijkstra_path_length(L_modes[mode], origin_node, weight=weight)
    
    # convert the dijkstra path costs dict to a dataframe like from R5
    tt_matrices[mode] = pd.DataFrame(list(dijkstra_path_costs.items()), columns=['to_node', f'travel_time_{mode}'])
    tt_matrices[mode]['from_node'] = origin_node
    
tt_matrices[MODE_PRIVATE_CARS]

private_cars
50608.0 28.9317621868226
foot
43277.0 28.9317621868226
cycling
43277.0 28.9317621868226


,to_node,travel_time_private_cars,from_node
0,50608.0,0.00,50608.0
1,52900.0,18.00,50608.0
2,50505.0,36.25,50608.0
3,50504.0,50.25,50608.0
4,50047.0,55.92,50608.0
...,...,...,...
53888,337.0,11402.70,50608.0
53889,27889.0,11571.55,50608.0
53890,26474.0,11629.76,50608.0
53891,32244.0,12115.60,50608.0


In [341]:
# merge the tt matrix of transit on cell_ids
statent_filtered_2 = pd.merge(
    statent_filtered, tt_matrices[MODE_TRANSIT],
    left_on='id', right_on='to_cell', how='left'
)
statent_filtered_2.drop(columns=['from_cell', 'to_cell'], inplace=True)
statent_filtered_2[f'travel_time_total_{MODE_TRANSIT}'] = statent_filtered_2[f'travel_time_{MODE_TRANSIT}']
statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry,closest_node_private_cars,access_egress_private_cars,closest_node_cycling,access_egress_cycling,closest_node_foot,access_egress_foot,travel_time_transit,travel_time_total_transit
0,2021,2023,2679100,1247300,67912473,6,0,0,6,59,...,6.935831,POINT (2679100.000 1247300.000),50608.0,28.931762,43277.0,28.931762,43277.0,28.931762,0.0,0.0
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,0.000000,POINT (2653100.000 1239900.000),24514.0,1358.115708,50566.0,241.644861,50566.0,241.644861,NaN,NaN
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,0.000000,POINT (2650500.000 1245800.000),53053.0,91.026213,51468.0,91.026213,51468.0,91.026213,6240.0,6240.0
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,4.000000,POINT (2690900.000 1282700.000),49794.0,1371.805307,24747.0,31.243944,24747.0,31.243944,7020.0,7020.0
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,4.000000,POINT (2637700.000 1255500.000),33074.0,51.306988,57357.0,51.306988,57357.0,51.306988,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248500,68082485,4,0,0,4,4,...,4.000000,POINT (2680800.000 1248500.000),43114.0,45.950393,75309.0,45.952784,75309.0,45.952784,1620.0,1620.0
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,4.000000,POINT (2714500.000 1258000.000),18198.0,1453.855424,37228.0,1352.131870,37228.0,1352.131870,6900.0,6900.0
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,4.000000,POINT (2696900.000 1228000.000),16756.0,25.600192,4735.0,25.600192,4735.0,25.600192,5040.0,5040.0
2936,2021,2023,2680700,1259300,68072593,4,4,0,0,4,...,0.000000,POINT (2680700.000 1259300.000),30881.0,130.510729,29692.0,130.510729,29692.0,130.510729,4680.0,4680.0


In [342]:
# merge the other tt matrices on node ids
for mode in [MODE_PRIVATE_CARS, MODE_FOOT, MODE_CYCLING]:

    # merge the travel time matrices with the statent dataset
    # such that every destination has the cost of reaching it
    statent_filtered_2 = pd.merge(
        statent_filtered_2, tt_matrices[mode],
        left_on=f'closest_node_{mode}', right_on='to_node', how='left'
    )
    statent_filtered_2.drop(columns=['from_node', 'to_node'], inplace=True)
    
    if mode == MODE_FOOT:
        speed_factor = walking_speed_kmh / 3.6
    elif mode == MODE_CYCLING:
        speed_factor = cycling_speed_kmh / 3.6
    else:
        speed_factor = 1
    
    # calculate total travel time by adding access and egress cost and considering speed factors
    statent_filtered_2[f'travel_time_total_{mode}'] = (
            (statent_filtered_2[f'travel_time_{mode}'] / speed_factor)
            + ((statent_filtered_2[f'access_egress_{mode}'] * access_egress_detour_factor) / (walking_speed_kmh / 3.6))
            + ((access_cost * access_egress_detour_factor) / (walking_speed_kmh / 3.6))
    )

statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,closest_node_foot,access_egress_foot,travel_time_transit,travel_time_total_transit,travel_time_private_cars,travel_time_total_private_cars,travel_time_foot,travel_time_total_foot,travel_time_cycling,travel_time_total_cycling
0,2021,2023,2679100,1247300,67912473,6,0,0,6,59,...,43277.0,28.931762,0.0,0.0,0.00,86.363469,0.000000,86.363469,0.000000,86.363469
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,50566.0,241.644861,NaN,NaN,3114.39,5184.610105,41148.322263,31111.548889,34666.041460,7337.053999
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,51468.0,91.026213,6240.0,6240.0,2995.83,3174.871754,38676.527405,29042.121908,30658.921318,6310.826018
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,24747.0,31.243944,7020.0,7020.0,3163.29,5253.942342,48151.931276,36024.091559,32348.887393,6559.591966
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,57357.0,51.306988,NaN,NaN,2879.26,2999.019329,56920.180180,42597.505732,43255.638471,8770.887023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248500,68082485,4,0,0,4,4,...,75309.0,45.952784,1620.0,1620.0,550.04,661.804410,2652.371288,2091.149537,1325.226420,376.813263
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,37228.0,1352.131870,6900.0,6900.0,3417.75,5630.865203,45602.857828,36093.272457,26243.463044,7309.981612
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,4735.0,25.600192,5040.0,5040.0,1977.89,2059.280976,29872.390988,22374.220072,16902.408268,3461.872630
2936,2021,2023,2680700,1259300,68072593,4,4,0,0,4,...,29692.0,130.510729,4680.0,4680.0,2411.18,2649.153868,16463.746414,12524.351789,9721.907768,2182.355421


In [343]:
statent_filtered_2['from_cell'] = statent_id
statent_filtered_2

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,access_egress_foot,travel_time_transit,travel_time_total_transit,travel_time_private_cars,travel_time_total_private_cars,travel_time_foot,travel_time_total_foot,travel_time_cycling,travel_time_total_cycling,from_cell
0,2021,2023,2679100,1247300,67912473,6,0,0,6,59,...,28.931762,0.0,0.0,0.00,86.363469,0.000000,86.363469,0.000000,86.363469,67912473
1,2021,2023,2653100,1239900,65312399,4,4,0,0,4,...,241.644861,NaN,NaN,3114.39,5184.610105,41148.322263,31111.548889,34666.041460,7337.053999,67912473
2,2021,2023,2650500,1245800,65052458,4,0,4,4,4,...,91.026213,6240.0,6240.0,2995.83,3174.871754,38676.527405,29042.121908,30658.921318,6310.826018,67912473
3,2021,2023,2690900,1282700,69092827,4,0,0,4,4,...,31.243944,7020.0,7020.0,3163.29,5253.942342,48151.931276,36024.091559,32348.887393,6559.591966,67912473
4,2021,2023,2637700,1255500,63772555,4,0,0,4,11,...,51.306988,NaN,NaN,2879.26,2999.019329,56920.180180,42597.505732,43255.638471,8770.887023,67912473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2021,2023,2680800,1248500,68082485,4,0,0,4,4,...,45.952784,1620.0,1620.0,550.04,661.804410,2652.371288,2091.149537,1325.226420,376.813263,67912473
2934,2021,2023,2714500,1258000,71452580,4,0,0,4,4,...,1352.131870,6900.0,6900.0,3417.75,5630.865203,45602.857828,36093.272457,26243.463044,7309.981612,67912473
2935,2021,2023,2696900,1228000,69692280,4,0,0,4,4,...,25.600192,5040.0,5040.0,1977.89,2059.280976,29872.390988,22374.220072,16902.408268,3461.872630,67912473
2936,2021,2023,2680700,1259300,68072593,4,4,0,0,4,...,130.510729,4680.0,4680.0,2411.18,2649.153868,16463.746414,12524.351789,9721.907768,2182.355421,67912473


In [344]:
destinations_with_cost = pd.merge(
    statpop_reduced.reset_index(),
    statent_filtered_2.reset_index()[[
        'id', 'VOLLZEITAEQ_TOTAL',
        'travel_time_total_cycling',
        'travel_time_total_foot',
        'travel_time_total_transit',
        'travel_time_total_private_cars',
        'access_egress_cycling',
        'access_egress_private_cars',
        'travel_time_private_cars',
        'closest_node_private_cars',
        'from_cell',
        'geometry'
    ]].rename(columns={'geometry': 'destination_geometry'}),
    how='left', left_on='statent_id', right_on='from_cell')
destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,travel_time_total_cycling,travel_time_total_foot,travel_time_total_transit,travel_time_total_private_cars,access_egress_cycling,access_egress_private_cars,travel_time_private_cars,closest_node_private_cars,from_cell,destination_geometry
0,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),67912473,35.135831,86.363469,86.363469,0.0,86.363469,28.931762,28.931762,0.00,50608.0,67912473,POINT (2679100.000 1247300.000)
1,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65312399,4.000000,7337.053999,31111.548889,NaN,5184.610105,241.644861,1358.115708,3114.39,24514.0,67912473,POINT (2653100.000 1239900.000)
2,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65052458,4.000000,6310.826018,29042.121908,6240.0,3174.871754,91.026213,91.026213,2995.83,53053.0,67912473,POINT (2650500.000 1245800.000)
3,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),69092827,4.000000,6559.591966,36024.091559,7020.0,5253.942342,31.243944,1371.805307,3163.29,49794.0,67912473,POINT (2690900.000 1282700.000)
4,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),63772555,6.989673,8770.887023,42597.505732,NaN,2999.019329,51.306988,51.306988,2879.26,33074.0,67912473,POINT (2637700.000 1255500.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8809,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68082485,4.000000,376.813263,2091.149537,1620.0,661.804410,45.952784,45.950393,550.04,43114.0,67912473,POINT (2680800.000 1248500.000)
8810,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),71452580,4.000000,7309.981612,36093.272457,6900.0,5630.865203,1352.131870,1453.855424,3417.75,18198.0,67912473,POINT (2714500.000 1258000.000)
8811,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),69692280,4.000000,3461.872630,22374.220072,5040.0,2059.280976,25.600192,25.600192,1977.89,16756.0,67912473,POINT (2696900.000 1228000.000)
8812,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68072593,4.000000,2182.355421,12524.351789,4680.0,2649.153868,130.510729,130.510729,2411.18,30881.0,67912473,POINT (2680700.000 1259300.000)


In [345]:
# add base travel time
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    destinations_with_cost[f'travel_time_total_{mode}'] += base_travel_time

In [346]:
# run the mode choice model to calculate each person's generalized cost to each statent destination

def calculate_behavioral_cost(private_cars=9999999, transit=9999999, cycling=9999999, foot=9999999, age=None):
    """
    Calculates individual generalized cost based on the personal properties and mode choice.
    Based on mode choice model in
    
    Hörl, S., M. Balać and K.W. Axhausen (2019)
    Pairing discrete mode choice models and agent-based transport simulation with MATSim,
    paper presented at the 98th Annual Meeting of the Transportation Research Board (TRB 2019),
    Washington DC, January 2019. 
    
    Returns a tuple of mode specific choice probabilities (dict) and the resulting behavioral travel time (float)

    Parameters
    ----------
    private_cars: float
        travel time in minutes
    transit: float
        travel time in minutes
    cycling: float
        travel time in minutes
    foot: float
        travel time in minutes
    age: int
        age in years

    Returns
    -------
    (dict, float)
    """
    
    # mode-specific costs (travel times in minutes)
    tt = {
        MODE_PRIVATE_CARS: private_cars,
        MODE_TRANSIT: transit,
        MODE_CYCLING: cycling,
        MODE_FOOT: foot
    }
    
    for mode, cost in tt.items():
        if np.isnan(cost) or np.isinf(cost):
            #tt[mode] = np.inf
            tt[mode] = 99999999
    
    # mode choice model parameters from Hörl et al. (2019), pp. 11, Table 1
    # t: time
    
    alpha_car =                    0.827   # [-]
    beta_tt_car =                 -0.0667  # [min-1]
    
    alpha_pt =                     0.0     # [-]
    beta_transfer =               -0.17    # [-]
    beta_in_vehicle_t =           -0.0192  # [min-1]
    beta_transfer_t =             -0.0384  # [min-1]
    beta_access_egress_t =        -0.0804  # [min-1]
    
    alpha_bike =                  -0.1     # [-]
    beta_tt_bike =                -0.0805  # [min-1]
    beta_age_bike =               -0.0496  # [years-1]
    
    alpha_walk =                   0.63    # [-]
    beta_tt_walk =                -0.141   # [min-1]
    
    beta_cost =                   -0.126   # [CHF-1]
    lambda_factor =               -0.4     # [-1]
    phi_avg_distance =            40.0     # [km]
    
    phi_parking_search_penalty =   6.0     # [min]
    phi_access_egress_walk_t =     5.0     # [min]
    
    # surrogate parameters to replace missing information (R5 provides only travel time)
    avg_speed_car =               25.0     # [kmh]
    detour_parameter_car =         1.3     # [-]
    x_crowfly_distance = tt[MODE_PRIVATE_CARS] / 60 * avg_speed_car / detour_parameter_car
    x_cost_car = tt[MODE_PRIVATE_CARS] * 0.2
    x_cost_pt = tt[MODE_TRANSIT] * 0.2
    
    x_transfers = round(tt[MODE_TRANSIT] / 30) if tt[MODE_TRANSIT] is not np.inf else np.inf
    x_transfer_t = x_transfers * 5
    x_access_egress_t = 5
    
    # mode-specific utilities from Hörl et al. (2019), pp. 10, eq. 3-6
    u = {
        MODE_PRIVATE_CARS: 
            alpha_car
            + beta_tt_car * tt[MODE_PRIVATE_CARS]
            + beta_tt_car * phi_parking_search_penalty
            + beta_tt_walk * phi_access_egress_walk_t
            + beta_cost * (x_crowfly_distance / phi_avg_distance)**lambda_factor * x_cost_car
        ,
        MODE_TRANSIT:
            alpha_pt
            + beta_transfer * x_transfers
            + beta_transfer_t * x_transfer_t
            + beta_access_egress_t * x_access_egress_t
            + beta_cost * (x_crowfly_distance / phi_avg_distance)**lambda_factor * x_cost_pt
        ,
        MODE_CYCLING:      
            alpha_bike
            + beta_tt_bike * tt[MODE_CYCLING]
            + beta_age_bike * max(0, age - 18)
        ,
        MODE_FOOT:
            alpha_walk
            + beta_tt_walk * tt[MODE_FOOT]
    }

    # mode-specific choice probabilities
    denominator = sum([np.exp(u_mode) for u_mode in u.values()])
    P = {mode: np.exp(u_mode) / denominator for mode, u_mode in u.items()}
    
    # total cost, after weighting the mode-specific cost by mode-specific choice probabilities
    tt_total = sum([tt[mode] * P[mode] for mode in tt.keys()])
    return P, tt_total

destinations_with_cost[['choice_probabilities', 'behavioral_cost']] = destinations_with_cost.apply(
    lambda row: calculate_behavioral_cost(
        private_cars=row['travel_time_total_private_cars'],
        transit=row['travel_time_total_transit'],
        foot=row['travel_time_total_foot'],
        cycling=row['travel_time_total_cycling'],
        age=row['age']
    ),
    axis=1,
    result_type='expand'
)

destinations_with_cost

C:\Users\lballo\AppData\Local\Temp\ipykernel_10272\2560101471.py:111: RuntimeWarning: invalid value encountered in scalar divide
  P = {mode: np.exp(u_mode) / denominator for mode, u_mode in u.items()}


,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,travel_time_total_transit,travel_time_total_private_cars,access_egress_cycling,access_egress_private_cars,travel_time_private_cars,closest_node_private_cars,from_cell,destination_geometry,choice_probabilities,behavioral_cost
0,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),67912473,35.135831,...,300.0,386.363469,28.931762,28.931762,0.00,50608.0,67912473,POINT (2679100.000 1247300.000),"{'private_cars': 6.799804415498586e-11, 'trans...",300.000000
1,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65312399,4.000000,...,NaN,5484.610105,241.644861,1358.115708,3114.39,24514.0,67912473,POINT (2653100.000 1239900.000),"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",5484.610105
2,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65052458,4.000000,...,6540.0,3474.871754,91.026213,91.026213,2995.83,53053.0,67912473,POINT (2650500.000 1245800.000),"{'private_cars': 3.425561798630964e-58, 'trans...",6540.000000
3,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),69092827,4.000000,...,7320.0,5553.942342,31.243944,1371.805307,3163.29,49794.0,67912473,POINT (2690900.000 1282700.000),"{'private_cars': 5.827025605093268e-119, 'tran...",7320.000000
4,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),63772555,6.989673,...,NaN,3299.019329,51.306988,51.306988,2879.26,33074.0,67912473,POINT (2637700.000 1255500.000),"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",3299.019329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8809,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68082485,4.000000,...,1920.0,961.804410,45.952784,45.950393,550.04,43114.0,67912473,POINT (2680800.000 1248500.000),"{'private_cars': 7.722917528421813e-14, 'trans...",1919.992854
8810,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),71452580,4.000000,...,7200.0,5930.865203,1352.131870,1453.855424,3417.75,18198.0,67912473,POINT (2714500.000 1258000.000),"{'private_cars': 8.86418168573514e-132, 'trans...",7200.000000
8811,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),69692280,4.000000,...,5340.0,2359.280976,25.600192,25.600192,1977.89,16756.0,67912473,POINT (2696900.000 1228000.000),"{'private_cars': 5.783113828105096e-31, 'trans...",5340.000000
8812,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68072593,4.000000,...,4980.0,2949.153868,130.510729,130.510729,2411.18,30881.0,67912473,POINT (2680700.000 1259300.000),"{'private_cars': 9.882152548534901e-54, 'trans...",4980.000000


In [347]:
"""
destinations_with_cost = pd.concat([
    destinations_with_cost,
    pd.json_normalize(destinations_with_cost['choice_probabilities']).add_prefix('prob_'),
    ],
    axis=1
)
"""

# calculate total accessibility contribution using cost function and destination utility
destinations_with_cost['accessibility_contribution'] = (
    (destinations_with_cost['behavioral_cost'] ** -0.7) 
    * destinations_with_cost['VOLLZEITAEQ_TOTAL']
)

destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,travel_time_total_private_cars,access_egress_cycling,access_egress_private_cars,travel_time_private_cars,closest_node_private_cars,from_cell,destination_geometry,choice_probabilities,behavioral_cost,accessibility_contribution
0,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),67912473,35.135831,...,386.363469,28.931762,28.931762,0.00,50608.0,67912473,POINT (2679100.000 1247300.000),"{'private_cars': 6.799804415498586e-11, 'trans...",300.000000,0.648284
1,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65312399,4.000000,...,5484.610105,241.644861,1358.115708,3114.39,24514.0,67912473,POINT (2653100.000 1239900.000),"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",5484.610105,0.009653
2,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65052458,4.000000,...,3474.871754,91.026213,91.026213,2995.83,53053.0,67912473,POINT (2650500.000 1245800.000),"{'private_cars': 3.425561798630964e-58, 'trans...",6540.000000,0.008534
3,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),69092827,4.000000,...,5553.942342,31.243944,1371.805307,3163.29,49794.0,67912473,POINT (2690900.000 1282700.000),"{'private_cars': 5.827025605093268e-119, 'tran...",7320.000000,0.007887
4,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),63772555,6.989673,...,3299.019329,51.306988,51.306988,2879.26,33074.0,67912473,POINT (2637700.000 1255500.000),"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",3299.019329,0.024076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8809,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68082485,4.000000,...,961.804410,45.952784,45.950393,550.04,43114.0,67912473,POINT (2680800.000 1248500.000),"{'private_cars': 7.722917528421813e-14, 'trans...",1919.992854,0.020126
8810,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),71452580,4.000000,...,5930.865203,1352.131870,1453.855424,3417.75,18198.0,67912473,POINT (2714500.000 1258000.000),"{'private_cars': 8.86418168573514e-132, 'trans...",7200.000000,0.007979
8811,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),69692280,4.000000,...,2359.280976,25.600192,25.600192,1977.89,16756.0,67912473,POINT (2696900.000 1228000.000),"{'private_cars': 5.783113828105096e-31, 'trans...",5340.000000,0.009835
8812,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68072593,4.000000,...,2949.153868,130.510729,130.510729,2411.18,30881.0,67912473,POINT (2680700.000 1259300.000),"{'private_cars': 9.882152548534901e-54, 'trans...",4980.000000,0.010328


In [348]:
# calculate accessibility contribution for every mode separately
for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
    destinations_with_cost[f'accessibility_contribution_{mode}'] = (
        (destinations_with_cost[f'travel_time_total_{mode}'] ** -0.7)
        * destinations_with_cost['VOLLZEITAEQ_TOTAL']
    )
    
destinations_with_cost

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,closest_node_private_cars,from_cell,destination_geometry,choice_probabilities,behavioral_cost,accessibility_contribution,accessibility_contribution_cycling,accessibility_contribution_private_cars,accessibility_contribution_transit,accessibility_contribution_foot
0,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),67912473,35.135831,...,50608.0,67912473,POINT (2679100.000 1247300.000),"{'private_cars': 6.799804415498586e-11, 'trans...",300.000000,0.648284,0.543066,0.543066,0.648284,0.543066
1,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65312399,4.000000,...,24514.0,67912473,POINT (2653100.000 1239900.000),"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",5484.610105,0.009653,0.007656,0.009653,NaN,0.002845
2,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),65052458,4.000000,...,53053.0,67912473,POINT (2650500.000 1245800.000),"{'private_cars': 3.425561798630964e-58, 'trans...",6540.000000,0.008534,0.008470,0.013286,0.008534,0.002984
3,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),69092827,4.000000,...,49794.0,67912473,POINT (2690900.000 1282700.000),"{'private_cars': 5.827025605093268e-119, 'tran...",7320.000000,0.007887,0.008254,0.009568,0.007887,0.002570
4,7921339,54,2,1,-2,-2,67912473.0,POINT (2679130.001 1247261.001),63772555,6.989673,...,33074.0,67912473,POINT (2637700.000 1255500.000),"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",3299.019329,0.024076,0.011860,0.024076,NaN,0.003997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8809,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68082485,4.000000,...,43114.0,67912473,POINT (2680800.000 1248500.000),"{'private_cars': 7.722917528421813e-14, 'trans...",1919.992854,0.020126,0.041757,0.032651,0.020126,0.017260
8810,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),71452580,4.000000,...,18198.0,67912473,POINT (2714500.000 1258000.000),"{'private_cars': 8.86418168573514e-132, 'trans...",7200.000000,0.007979,0.007675,0.009139,0.007979,0.002567
8811,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),69692280,4.000000,...,16756.0,67912473,POINT (2696900.000 1228000.000),"{'private_cars': 5.783113828105096e-31, 'trans...",5340.000000,0.009835,0.012568,0.017423,0.009835,0.003574
8812,12659585,67,2,2,-2,-2,67912473.0,POINT (2679103.001 1247285.001),68072593,4.000000,...,30881.0,67912473,POINT (2680700.000 1259300.000),"{'private_cars': 9.882152548534901e-54, 'trans...",4980.000000,0.010328,0.016813,0.014903,0.010328,0.005326


In [349]:
# for each person, sum the accessibility contributions across all destinations
accessibility = destinations_with_cost.groupby('record').agg({
    'age': 'first',
    'sex': 'first',
    'maritalstatus': 'first',
    'residencepermit': 'first',
    'residentpermit': 'first',
    'statent_id': 'first',
    'accessibility_contribution': 'sum',
    'accessibility_contribution_cycling': 'sum',
    'accessibility_contribution_foot': 'sum',
    'accessibility_contribution_private_cars': 'sum',
    'accessibility_contribution_transit': 'sum',
    'geometry': 'first'
})

accessibility.rename(columns={
    'accessibility_contribution': 'accessibility',
    'accessibility_contribution_cycling': 'accessibility_cycling',
    'accessibility_contribution_foot': 'accessibility_foot',
    'accessibility_contribution_private_cars': 'accessibility_private_cars',
    'accessibility_contribution_transit': 'accessibility_transit',
}, inplace=True)

accessibility = gpd.GeoDataFrame(accessibility, crs=2056)
accessibility

,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility,accessibility_cycling,accessibility_foot,accessibility_private_cars,accessibility_transit,geometry
record,,,,,,,,,,,,
7921339,54,2,1,-2,-2,67912473.0,231.635819,363.183619,126.851578,320.075892,218.761534,POINT (2679130.001 1247261.001)
9180957,65,1,1,-2,-2,67912473.0,231.635429,363.183619,126.851578,320.075892,218.761534,POINT (2679102.001 1247322.001)
12659585,67,2,2,-2,-2,67912473.0,231.635361,363.183619,126.851578,320.075892,218.761534,POINT (2679103.001 1247285.001)


In [350]:
if 0:
    snman.io.export_gdf(
        accessibility,
        os.path.join(outputs_path, f'accessibility_{STATE}.gpkg')
    )

In [351]:
accessibility_contributions = copy.deepcopy(destinations_with_cost)
accessibility_contributions['geometry'] = accessibility_contributions.apply(
    lambda row: shp.LineString([row['geometry'], row['destination_geometry']]),
    axis=1
)
accessibility_contributions['geometry'] = accessibility_contributions['destination_geometry']
accessibility_contributions.drop(columns=['destination_geometry'], inplace=True)

snman.io.export_gdf(
    accessibility_contributions,
    os.path.join(outputs_path, f'accessibility_contributions_{STATE}.gpkg')
)

accessibility_contributions

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry,id,VOLLZEITAEQ_TOTAL,...,travel_time_private_cars,closest_node_private_cars,from_cell,choice_probabilities,behavioral_cost,accessibility_contribution,accessibility_contribution_cycling,accessibility_contribution_private_cars,accessibility_contribution_transit,accessibility_contribution_foot
0,7921339,54,2,1,-2,-2,67912473.0,POINT (2679100.000 1247300.000),67912473,35.135831,...,0.00,50608.0,67912473,"{'private_cars': 6.799804415498586e-11, 'trans...",300.000000,0.648284,0.543066,0.543066,0.648284,0.543066
1,7921339,54,2,1,-2,-2,67912473.0,POINT (2653100.000 1239900.000),65312399,4.000000,...,3114.39,24514.0,67912473,"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",5484.610105,0.009653,0.007656,0.009653,NaN,0.002845
2,7921339,54,2,1,-2,-2,67912473.0,POINT (2650500.000 1245800.000),65052458,4.000000,...,2995.83,53053.0,67912473,"{'private_cars': 3.425561798630964e-58, 'trans...",6540.000000,0.008534,0.008470,0.013286,0.008534,0.002984
3,7921339,54,2,1,-2,-2,67912473.0,POINT (2690900.000 1282700.000),69092827,4.000000,...,3163.29,49794.0,67912473,"{'private_cars': 5.827025605093268e-119, 'tran...",7320.000000,0.007887,0.008254,0.009568,0.007887,0.002570
4,7921339,54,2,1,-2,-2,67912473.0,POINT (2637700.000 1255500.000),63772555,6.989673,...,2879.26,33074.0,67912473,"{'private_cars': 1.0, 'transit': 0.0, 'cycling...",3299.019329,0.024076,0.011860,0.024076,NaN,0.003997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8809,12659585,67,2,2,-2,-2,67912473.0,POINT (2680800.000 1248500.000),68082485,4.000000,...,550.04,43114.0,67912473,"{'private_cars': 7.722917528421813e-14, 'trans...",1919.992854,0.020126,0.041757,0.032651,0.020126,0.017260
8810,12659585,67,2,2,-2,-2,67912473.0,POINT (2714500.000 1258000.000),71452580,4.000000,...,3417.75,18198.0,67912473,"{'private_cars': 8.86418168573514e-132, 'trans...",7200.000000,0.007979,0.007675,0.009139,0.007979,0.002567
8811,12659585,67,2,2,-2,-2,67912473.0,POINT (2696900.000 1228000.000),69692280,4.000000,...,1977.89,16756.0,67912473,"{'private_cars': 5.783113828105096e-31, 'trans...",5340.000000,0.009835,0.012568,0.017423,0.009835,0.003574
8812,12659585,67,2,2,-2,-2,67912473.0,POINT (2680700.000 1259300.000),68072593,4.000000,...,2411.18,30881.0,67912473,"{'private_cars': 9.882152548534901e-54, 'trans...",4980.000000,0.010328,0.016813,0.014903,0.010328,0.005326


In [354]:
# testbed for point to point routing

# node IDs
#origin, destination = (1756, 51785) # cross-city before
origin, destination = (50608, 1776) # cross-city after
#origin, destination = (50608, 35180)
weight = 'matsim_tt_cars'

L = L_modes[MODE_PRIVATE_CARS]
route = nx.shortest_path(L, origin, destination, weight=weight)

uvk_and_order = {}
for u, v in zip(route[:-1], route[1:]):
    k, data = min(L.get_edge_data(u, v).items(), key=lambda e: e[1][weight] + (e[1]['lane'].lanetype == LANETYPE_MOTORIZED))
    uvk_and_order[(u,v,k)] = len(uvk_and_order)
    
M = L.edge_subgraph(uvk_and_order.keys())
nx.set_edge_attributes(M, uvk_and_order, 'order_in_path')
M = snman.lane_graph.LaneGraph(M)

edges = oxc.graph_to_gdfs(M, nodes=False)
print(edges.groupby('lanetype').agg({weight: 'sum', 'length': 'sum'}))
print(edges.agg({weight: 'sum', 'length': 'sum'}))

snman.io.export_lane_geometries(
    M,
    os.path.join(outputs_path, f'path_edges.gpkg'),
    os.path.join(outputs_path, f'path_nodes.gpkg'),
    scaling=30
)

          matsim_tt_cars        length
lanetype                              
M                 2642.7  13220.656047
matsim_tt_cars     2642.700000
length            13220.656047
dtype: float64


In [353]:
STATE

'after'